In [ ]:
!pip install tensorflowjs

import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflowjs as tfjs
from sklearn.metrics import confusion_matrix

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Version:  2.8.2
Eager mode:  True
Hub version:  0.12.0
GPU is available


Descargar los datos

In [ ]:
print(f'data imdb')
#descargar todos, pero es muy pesado :(
#(train_data, train_labels), (test_data, test_labels) = tfds.load(name="imdb_reviews", split=('train[:60%]', 'train[60%:]', 'test'), as_supervised=True,num_words=10000)


from tensorflow import keras
data=keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words=10000)
#(train_data, trainlabels), (test_data, testlabels) = data.load_data(num_words=10000)
#dicc={0:'Negativa',1:'Positiva'}

#train_labels=tf.keras.utils.to_categorical(trainlabels)
#test_labels=tf.keras.utils.to_categorical(testlabels)
print(len(train_data), len(test_data))  
#print(f'{train_labels[0]}')
#print(f'{train_data[0]}-{train_labels[0]}')
#print(f'{test_data[0]}-{test_labels[0]}')


data imdb


25000 25000


Explorar los datos

In [ ]:
# 0| negativa
# 1| positiva
print(train_data[0])
print(train_labels[0])


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1


In [ ]:
#index the words 
word_index = data.get_word_index() #get the word index
#print(word_index)#tuple of (word, index)
word_index = {k:(v+3) for k,v in word_index.items()} #add 3 to all the values
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2
word_index["<UNUSED>"] = 3

reverse_word_index=dict([(value,key) for (key,value) in word_index.items()]) #voltear de key a value a value key

#print(len(train_data[0])-len(train_data[2]))
train_data=keras.preprocessing.sequence.pad_sequences(
    train_data, 
    value=word_index["<PAD>"],
    padding="post",
    maxlen=256
    )
test_data=keras.preprocessing.sequence.pad_sequences(
    test_data, 
    value=word_index["<PAD>"],
    padding="post",
    maxlen=256
    )
#print(len(train_data[0])-len(train_data[2]))

def decode_review(text):
    return " ".join([reverse_word_index.get(i,"?") for i in text])


Modelo red neuronal 

In [ ]:
def create_model():
  #modelo
  model=tf.keras.models.Sequential([
        tf.keras.layers.Embedding(10000,16),#los vectores creo
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(16, activation='relu'), #hidden layer
        tf.keras.layers.Dropout(0.2),#dropout layer
        tf.keras.layers.Dense(1, activation='sigmoid')]) #outputs probability between 0 and 1  
        #tf.keras.layers.Dropout(0.2),#dropout layer
        #tf.keras.layers.Dense(1, activation='softmax')]) #outputs an array
  
  learning_rate=.001
  #opt=tf.keras.optimizers.SGD(learning_rate=learning_rate)
  opt='adam'
  #compile
  model.compile(optimizer=opt,
            loss="binary_crossentropy",
            #loss='categorical_crossentropy',
            #loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

  return model

modelo rnn


In [ ]:
def create_modelRNN():
  #modelo
  model=tf.keras.models.Sequential([
        tf.keras.layers.Embedding(10000,16),#los vectores creo
        
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),#dropout layer
        tf.keras.layers.Dense(1, activation='sigmoid')]) #outputs probability between 0 and 1     
        #tf.keras.layers.Dense(1, activation='softmax') #outputs probability between 0 and 1 
  
  learning_rate=.001
  #opt=tf.keras.optimizers.SGD(learning_rate=learning_rate)
  opt='adam'
  #compile
  model.compile(optimizer=opt,
            loss="binary_crossentropy",
            #loss='categorical_crossentropy',
            #loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

  return model

crear el modelo



In [ ]:
model=create_model()
#model=create_modelRNN()
model.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, None, 16)          160000    
                                                                 
 global_average_pooling1d_14  (None, 16)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_24 (Dense)            (None, 16)                272       
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                                 
 dense_25 (Dense)            (None, 1)                 17        
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_______________________________________________

# Nueva sección

Entrenar y guardar el modelo 

In [ ]:
n_epochs=30
n_batch=32

#path
checkpoint_path='trainint_1/cp.ckpt'
checkpoint_dir=os.path.dirname(checkpoint_path)
cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,verbose=1)

#entrenar
H=model.fit(train_data, train_labels, epochs=n_epochs,batch_size=n_batch,callbacks=[cp_callback])

#evaluar la exactitud
test_loss,test_acc=model.evaluate(test_data, test_labels, verbose=2)
print(f'\nTest accuracy: {test_acc:.2f}')      


# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))


Epoch 1/30
773/782 [============================>.] - ETA: 0s - loss: 0.5623 - accuracy: 0.7170
Epoch 1: saving model to trainint_1/cp.ckpt
782/782 [==============================] - 4s 4ms/step - loss: 0.5599 - accuracy: 0.7187
Epoch 2/30
778/782 [============================>.] - ETA: 0s - loss: 0.2968 - accuracy: 0.8844
Epoch 2: saving model to trainint_1/cp.ckpt
782/782 [==============================] - 3s 4ms/step - loss: 0.2965 - accuracy: 0.8846
Epoch 3/30
773/782 [============================>.] - ETA: 0s - loss: 0.2328 - accuracy: 0.9119
Epoch 3: saving model to trainint_1/cp.ckpt
782/782 [==============================] - 3s 4ms/step - loss: 0.2327 - accuracy: 0.9121
Epoch 4/30
773/782 [============================>.] - ETA: 0s - loss: 0.1986 - accuracy: 0.9279
Epoch 4: saving model to trainint_1/cp.ckpt
782/782 [==============================] - 3s 4ms/step - loss: 0.1983 - accuracy: 0.9280
Epoch 5/30
780/782 [============================>.] - ETA: 0s - loss: 0.1728 - accur

Cargar el modelo 

In [ ]:
model=create_model()
#model=create_modelRNN()

#load the weights
checkpoint_path='trainint_1/cp.ckpt'
model.load_weights(checkpoint_path)

predictions=model.predict(test_data)

for i in range(len(predictions)):
  if predictions[i]>.5:
    print('Positiva')
  else:
    print('Negativa')
  


Se han truncado las últimas 5000 líneas del flujo de salida.
Negativa
Negativa
Positiva
Negativa
Positiva
Negativa
Negativa
Positiva
Positiva
Negativa
Positiva
Positiva
Negativa
Positiva
Positiva
Positiva
Negativa
Negativa
Negativa
Positiva
Positiva
Positiva
Positiva
Negativa
Positiva
Negativa
Positiva
Positiva
Negativa
Negativa
Positiva
Negativa
Negativa
Positiva
Negativa
Positiva
Positiva
Negativa
Negativa
Negativa
Negativa
Positiva
Negativa
Negativa
Negativa
Negativa
Positiva
Negativa
Negativa
Positiva
Negativa
Positiva
Positiva
Positiva
Negativa
Negativa
Positiva
Negativa
Positiva
Negativa
Positiva
Negativa
Negativa
Positiva
Positiva
Positiva
Negativa
Positiva
Negativa
Negativa
Positiva
Positiva
Negativa
Positiva
Positiva
Positiva
Positiva
Negativa
Positiva
Positiva
Positiva
Negativa
Negativa
Positiva
Negativa
Positiva
Positiva
Negativa
Positiva
Positiva
Positiva
Negativa
Negativa
Negativa
Negativa
Negativa
Positiva
Positiva
Positiva
Negativa
Negativa
Positiva
Negativa
Negativa
Neg

# Nueva sección